In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import torch
import torchvision
from torchvision import datasets
from torchvision import transforms as T # for simplifying the transforms
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader, sampler, random_split
from torchvision import models
import sys
from tqdm import tqdm
import time
import copy
import matplotlib.pyplot as plt
%matplotlib inline
!pip install timm 
import timm
from timm.loss import LabelSmoothingCrossEntropy # This is better than normal nn.CrossEntropyLoss
from torch.nn import CrossEntropyLoss
import warnings
warnings.filterwarnings("ignore")
!pip install split-folders
import splitfolders
!pip install opencv-python
import cv2
from PIL import Image
!pip install efficientnet_pytorch
from efficientnet_pytorch import EfficientNet
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 549 kB 14.4 MB/s 
     |████████████████████████████████| 182 kB 72.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=9ecc3e0024561eaf2a7db34804b94cfd04484f149d0afdd53d10b752f5ce7bc3
  Stored in directory: /root/.cache/pip/wheels/84/b9/90/25a0195cf95fb5533db96f1c77ea3f296b7cc86ae8ae48e3dc
Successfully built efficientnet-pytorch
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 15.1 MB/s 
     |

In [ ]:
import wandb
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
PROJECT = 'homework2_ML'
DATA_DIR = 'data'
ARTIFACT_PATH = '/content/w&b/'

#Utilities for dataset manipulation

In [ ]:
def get_classes(data_dir):
    all_data = datasets.ImageFolder(data_dir)
    return all_data.classes

In [ ]:
def get_data_loaders(data_dir, batch_size, train = False):
    if train:
        #train
        transform = T.Compose([
            T.RandomHorizontalFlip(),
            T.RandomVerticalFlip(),
            T.RandomApply(torch.nn.ModuleList([T.ColorJitter()]), p=0.25),
            T.Resize(256),
            T.CenterCrop(224),
            T.ToTensor(),
            T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)), # imagenet means
            T.RandomErasing(p=0.2, value='random')
        ])
        train_data = datasets.ImageFolder(os.path.join(data_dir, "train/"), transform = transform)
        train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4)
        return train_loader, len(train_data)
    else:
        # val/test
        transform = T.Compose([ # We dont need augmentation for test transforms
            T.Resize(256),
            T.CenterCrop(224),
            T.ToTensor(),
            T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)), # imagenet means
        ])
        val_data = datasets.ImageFolder(os.path.join(data_dir, "valid/"), transform=transform)
        test_data = datasets.ImageFolder(os.path.join(data_dir, "test/"), transform=transform)
        val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True, num_workers=4)
        test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=4)
        return val_loader, test_loader, len(val_data), len(test_data)

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
dataset_path_original = "/content/drive/MyDrive/ColabNotebooks/MachineLearning/HomeWork2/Dataset"
splitfolders.ratio(dataset_path_original, output="dataset_", seed=1337, ratio=(.8, 0.1,0.1)) 
classes = get_classes('/content/dataset_/train/')
print("Found : ",len(classes),"classes --->>>",classes)

Copying files: 2911 files [00:35, 82.42 files/s] 

Found :  10 classes --->>> ['Balcony', 'Bathroom', 'Bedroom', 'Fireplace', 'Garden', 'Hammok', 'Kitchen', 'Panoramic_view', 'Pool', 'Stairs']


In [ ]:
dataset_path = "/content/dataset_"
batch_size = 32
training_batch_size = 128

In [ ]:
#Rename the folder val in /dataset_ from val to valid
(train_loader, train_data_len) = get_data_loaders(dataset_path, training_batch_size , train=True)
(val_loader, test_loader, valid_data_len, test_data_len) = get_data_loaders(dataset_path, batch_size=batch_size, train=False)

In [ ]:
dataloaders = {
    "train": train_loader,
    "val": val_loader
}
dataset_sizes = {
    "train": train_data_len,
    "val": valid_data_len
}

In [ ]:
print("Batch size (val) = ", batch_size)
print("Batch size (training) = ", training_batch_size)
print("Batches per training: ", len(train_loader))
print("Batches per validation: ", len(val_loader))
print("Batches per test: ", len (test_loader) )

Batch size (val) =  32
Batch size (training) =  128
Batches per training:  19
Batches per validation:  9
Batches per test:  10


In [ ]:
print(train_data_len, valid_data_len, test_data_len)

2324 287 300


#Log dati a Wandb

Log alcune immagini del dataset

In [ ]:
wandb.init(project=PROJECT , job_type="log_dataset")
table = wandb.Table(columns=["image", "label"])
for step, (img, label) in enumerate(tqdm(train_loader, leave=False)):
    w_img = wandb.Image(np.transpose(img[0].numpy(), axes=(1,2,0)))
    table.add_data(w_img, classes[label.numpy()[0]])
    if step == 10: 
        break


wandb.log({'train_data/train_table':table})
wandb.finish()

wandb: Currently logged in as: rucci-2053183 (sapienza_ml_2022_23). Use `wandb login --relogin` to force relogin


Log distribuzione dei batches

In [ ]:
from collections import Counter
dat = datasets.ImageFolder(dataset_path_original)
classes = []
distrib = {}

for item in dat.class_to_idx.keys():
  classes.append(item)

for i,item in enumerate(dict(Counter(dat.targets)).values()):
  distrib[classes[i]] = item

distrib

{'Balcony': 218,
 'Bathroom': 300,
 'Bedroom': 306,
 'Fireplace': 266,
 'Garden': 321,
 'Hammok': 203,
 'Kitchen': 321,
 'Panoramic_view': 333,
 'Pool': 355,
 'Stairs': 288}

In [ ]:
wandb.init(project=PROJECT , job_type="dataset_distribution")
wandb.log({'data_distrib': distrib})
wandb.finish()

#Scelta del Modello

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Flying on the ->",device)

Flying on the -> cuda


In [ ]:
config = {
    "learning_rate": 0.001,
    "num_epochs": 300,
    "batch_size": 32,
    "training_batch_size": 128,
    "img_size": (224, 224, 3),
    'backbone': 'efficientNet'
}

In [ ]:
LR = config['learning_rate']
EPOCHS = config['num_epochs']

# Log the final results on the validation set
LOG_IMAGES = True
LOG_CONFUSION_MATRIX = True





In [ ]:
def build_model(num_classes=10, backbone='deit'):
    if backbone == 'deit':
      model = torch.hub.load('facebookresearch/deit:main', 'deit_tiny_patch16_224', pretrained=True)
      for param in model.parameters(): #freeze model
        param.requires_grad = False

      n_inputs = model.head.in_features
      model.head = nn.Sequential(
          nn.Linear(n_inputs, 512),
          nn.ReLU(),
          nn.Dropout(0.3),
          nn.Linear(512, len(classes))
      )
      model = model.to(device)
    
    if backbone == 'efficientNet':
      model = EfficientNet.from_pretrained('efficientnet-b0')
      n_inputs = model.head.in_features
      model.head = nn.Sequential(
          nn.Linear(n_inputs, 512),
          nn.ReLU(),
          nn.Dropout(0.3),
          nn.Linear(512, len(classes))
      )
      model = model.to(device)

      
    return model


In [ ]:
model = build_model(backbone=config['backbone'])
criterion = LabelSmoothingCrossEntropy()
criterion = criterion.to(device)
optimizer = optim.Adam(model.parameters(), config['learning_rate'])

Loaded pretrained weights for efficientnet-b0


In [ ]:
# lr scheduler
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.97)

Inizializzazione Wandb

In [ ]:
wandb.init(project=PROJECT, sync_tensorboard=True, job_type='training_logs')
wandb.config.update(config)

wandb: Currently logged in as: rucci-2053183 (sapienza_ml_2022_23). Use `wandb login --relogin` to force relogin


In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=config['num_epochs'],callbacks=None):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    wandb.watch(model, log_freq=10)

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print("-"*10)
        
        for phase in ['train', 'val']: # We do training and validation phase per epoch
            if phase == 'train':
                
                model.train() # model to training mode
            else:
                model.eval() # model to evaluate
            
            running_loss = 0.0
            running_corrects = 0.0
            
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'): # no autograd makes validation go faster
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1) # used for accuracy
                    loss = criterion(outputs, labels)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            if phase == 'train':
                scheduler.step() # step at end of epoch
            
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc =  running_corrects.double() / dataset_sizes[phase]
            if epoch % 10 == 0: #ogi 10 epoche printo su wandb
              wandb.log({phase+"_loss": epoch_loss})
              wandb.log({phase+"_acc": epoch_acc})

            
            print("{} Loss: {:.4f} Acc: {:.4f}".format(phase, epoch_loss, epoch_acc))
            
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict()) # keep the best validation accuracy model
        print()
    time_elapsed = time.time() - since # slight error
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print("Best Val Acc: {:.4f}".format(best_acc))
    
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
model_ft = train_model(model, criterion, optimizer, exp_lr_scheduler,callbacks=None)
wandb.finish()

Epoch 0/299
----------


100%|██████████| 19/19 [00:25<00:00,  1.36s/it]


train Loss: 3.4833 Acc: 0.4836


100%|██████████| 9/9 [00:01<00:00,  5.12it/s]


val Loss: 2.8613 Acc: 0.7561

Epoch 1/299
----------


...

100%|██████████| 19/19 [00:18<00:00,  1.02it/s]


train Loss: 1.0621 Acc: 0.9966


100%|██████████| 9/9 [00:01<00:00,  6.13it/s]


val Loss: 1.3322 Acc: 0.9094

Epoch 37/299
----------


 68%|██████▊   | 13/19 [00:13<00:05,  1.12it/s]

In [ ]:
example = torch.rand(1, 3, 224, 224)
traced_script_module = torch.jit.trace(model.cpu(), example)
# traced_script_module.save("VisionTransformer_DEIT_Facebook.pt")
traced_script_module.save("EfficientNet_b0.pt")
# traced_script_module.save("/content/drive/MyDrive/ColabNotebooks/MachineLearning/HomeWork2/VisionTransformer_DEIT_Facebook.pt")
traced_script_module.save("/content/drive/MyDrive/ColabNotebooks/MachineLearning/HomeWork2/EfficientNet_b0.pt")

#Import the model 


In [ ]:
#Deit
model = torch.load("/content/drive/MyDrive/ColabNotebooks/MachineLearning/HomeWork2/VisionTransformer_DEIT_Facebook.pt")

#Testing